## Importing Relevant Libraries for WebScraping and Setting up the DataFrame

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### I use BS to scrape the table from wikipedia

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_ = "wikitable sortable").tbody.text

### The scraped table information is in str format. Here I convert it to a Pandas Dataframe

In [3]:
table_list = [line.split('\n') for line in (table.split('\n\n'))]

df = pd.DataFrame(table_list, columns = table_list[0])
df.drop([0,289], inplace= True)#I drop the first and last rows which are the table header and none, repectively

df.head()

,,Postcode,Borough,Neighbourhood
1,,M1A,Not assigned,Not assigned
2,,M2A,Not assigned,Not assigned
3,,M3A,North York,Parkwoods
4,,M4A,North York,Victoria Village
5,,M5A,Downtown Toronto,Harbourfront


### Table rows with unassigned Boroughs are dropped

In [4]:
na = df['Borough'] == "Not assigned"
df = df[~na].reset_index(drop = True)

In [5]:
df.shape

(211, 4)

In [6]:
df.head(10)

,,Postcode,Borough,Neighbourhood
0,,M3A,North York,Parkwoods
1,,M4A,North York,Victoria Village
2,,M5A,Downtown Toronto,Harbourfront
3,,M5A,Downtown Toronto,Regent Park
4,,M6A,North York,Lawrence Heights
5,,M6A,North York,Lawrence Manor
6,,M7A,Queen's Park,Not assigned
7,,M9A,Etobicoke,Islington Avenue
8,,M1B,Scarborough,Rouge
9,,M1B,Scarborough,Malvern


### I replace unassigned Neighborhoods with the name of the Borough

In [7]:
df['Neighbourhood'][df['Neighbourhood'] == "Not assigned"] = df['Borough'][df['Neighbourhood'] == "Not assigned"]

In [8]:
df.head(10)

,,Postcode,Borough,Neighbourhood
0,,M3A,North York,Parkwoods
1,,M4A,North York,Victoria Village
2,,M5A,Downtown Toronto,Harbourfront
3,,M5A,Downtown Toronto,Regent Park
4,,M6A,North York,Lawrence Heights
5,,M6A,North York,Lawrence Manor
6,,M7A,Queen's Park,Queen's Park
7,,M9A,Etobicoke,Islington Avenue
8,,M1B,Scarborough,Rouge
9,,M1B,Scarborough,Malvern


### Here, I aggregate the df based on Neighborhoods with similar Boroughs and Postcodes

In [9]:
df_grouped = df.groupby(['Postcode', 'Borough'], as_index = 0, sort = 0).agg(', '.join)

In [10]:
df_grouped.head()

,Postcode,Borough,,Neighbourhood
0,M3A,North York,,Parkwoods
1,M4A,North York,,Victoria Village
2,M5A,Downtown Toronto,",","Harbourfront, Regent Park"
3,M6A,North York,",","Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,,Queen's Park


In [11]:
df_grouped.shape

(103, 4)

### Read in the CSV file with the Geospatial Coordinates of Toronto

In [16]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [17]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_coordinates.rename(columns = {'Postal Code': 'Postcode'}, inplace=True)
df_coordinates.head(2)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


### Merge the Neighborhood df with the Geospatial df

In [21]:
df_complete = df_grouped.merge(df_coordinates, how = 'left', on = 'Postcode')

### Checking for any null values

In [23]:
pd.isnull(df_complete['Latitude']).sum()

0

In [24]:
df_complete.sample(10)

,Postcode,Borough,,Neighbourhood,Latitude,Longitude
24,M5G,Downtown Toronto,,Central Bay Street,43.657952,-79.387383
62,M5N,Central Toronto,,Roselawn,43.711695,-79.416936
85,M1V,Scarborough,", , ,","Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
23,M4G,East York,,Leaside,43.709060,-79.363452
2,M5A,Downtown Toronto,",","Harbourfront, Regent Park",43.654260,-79.360636
88,M8V,Etobicoke,", ,","Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
18,M1E,Scarborough,", ,","Guildwood, Morningside, West Hill",43.763573,-79.188711
70,M9P,Etobicoke,,Westmount,43.696319,-79.532242
3,M6A,North York,",","Lawrence Heights, Lawrence Manor",43.718518,-79.464763
71,M1R,Scarborough,",","Maryvale, Wexford",43.750072,-79.295849
